In [2]:
pip install clean-text
# Hej Karen

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\karen\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\karen\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [124]:
#################################### PACKAGES ######################################
import pandas as pd
from cleantext import clean
import re

import nltk
from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize 
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import *


R = pd.read_csv("https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv") 


####################################### CLEANING ######################################
Copy_clean = R.copy()

def cleaningTime(x):
    clean_text = x.replace('|', '')
    clean_text = clean_text.replace('$', '')
    regex = r'[xX]+'
    clean_text = re.sub(regex, '', clean_text)
    #clean_text = x.replace(',', '')
    clean_text = clean(clean_text,
        no_punct=True,
        lower=True,                    # lowercase text
        no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
        no_urls=True,                  # replace all URLs with a special token
        no_emails=True,                # replace all email addresses with a special token
        no_numbers=True,
        replace_with_url="URL",
        replace_with_email="EMAIL",
        replace_with_number="NUMBER",
        )
    return clean_text

#R['content'].apply(cleaning)
Copy_clean['content'] = R['content'].apply(cleaningTime)
#print(Copy_clean['content'])
df = pd.DataFrame(Copy_clean)
df.to_csv(r'C:\Users\karen\nein.csv', index= None, header=True)


####################################### TOKENIZE #########################################
Copy_tok = Copy_clean.copy()

def Tokenize(x):     
    tokens = nltk.word_tokenize(x)
    return tokens

Copy_tok['content'] = Copy_tok['content'].apply(Tokenize)
print(Copy_tok['content'])

#Copy_stop = []
#for i in Copy_tok['content']:
    #tokens = nltk.word_tokenize(i)
    #Copy_stop.append(tokens)
#print(Copy_stop)

######################################## REMOVING STOPWORDS ################################
Copy_stop = Copy_tok.copy()
stop_words = set(stopwords.words('english'))
print(len(stop_words))

# removing
def Remove_stopwords(x): # x = Copy_tok['content']
    filtered_sentence = []
    for w in x:
        if w not in stop_words:
            filtered_sentence.append(w)
    return filtered_sentence

Copy_stop['content'] = Copy_stop['content'].apply(Remove_stopwords)
#print(Copy_stop['content'])


######################################### STEMMING ##########################################
Copy_stemming = Copy_stop.copy()
stemmer = SnowballStemmer("english")

def Stemming_Words(x):
    stem_words = []
    for w in x:
        word = stemmer.stem(w)
        stem_words.append(word)
    return stem_words
Copy_stemming['content'] = Copy_stemming['content'].apply(Stemming_Words)
#print(Copy_stemming['content'])

##############################################################################################
# number of tokens before removing stopwords
def unique_words (file):
    UN = {}
    for row in file:
        for word in row:
            if word not in UN:
                UN[word] = 1
            else: 
                UN[word] += 1
    UN = dict(sorted(UN.items(), key = lambda kv: kv[1], reverse=True)) # Sorting the unique words after number of occurrences, from highest to lowest
    return len(UN)

print(unique_words(Copy_tok['content']))
####
print(unique_words(Copy_tok['content'])-unique_words(Copy_stop['content']))

# reductionrate - from token to after removing stopwords 
x = 100 - 96127/169480 * 100 # = 43.28%
#print(x)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\karen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0      [sometimes, the, power, of, christmas, will, m...
1      [awakening, of, number, strands, of, dna, reco...
2      [never, hike, alone, a, friday, the, 13th, fan...
3      [when, a, rare, shark, was, caught, scientists...
4      [donald, trump, has, the, unnerving, ability, ...
                             ...                        
245    [prison, for, rahm, gods, work, and, many, oth...
246    [number, useful, items, for, your, tiny, home,...
247    [former, cia, director, michael, hayden, said,...
248    [antonio, sabato, jr, says, hollywoods, libera...
249    [former, us, president, bill, clinton, on, mon...
Name: content, Length: 250, dtype: object
179
16641
133


In [125]:

# COUNTING
count = 0 
wordlist = ['url', 'number']
for i in Copy_stemming['content']:
    for x in i:
        if x in wordlist:
            count += 1
        else:
            count = count
#count